# Module 3 Project - Web Scraping

In [15]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import json
import time

In [2]:
%%capture

from tqdm import tqdm_notebook as tqdm
from tqdm import tnrange
tqdm().pandas()

In [81]:
links = []
url = 'https://distiller.com/search?page=1'

for i in tqdm(range(357)):
    
    html_page = requests.get(url) 
    soup = BeautifulSoup(html_page.content, 'html.parser')

    for content in soup.findAll('li', {'class': 'whiskey-content'}):
        for whisky in content.findAll('a', href=True):
            links.append(whisky['href'])
    
    url = soup.find('a', {'rel': 'next'})
    url = 'https://distiller.com/search?' + url['href']

In [84]:
links

['/spirits/hibiki-21-year',
 '/spirits/highland-park-18',
 '/spirits/george-t-stagg-bourbon-fall-2019',
 '/spirits/bowmore-mizunara-cask-finish',
 '/spirits/the-macallan-m',
 '/spirits/port-ellen-32-year-1979-2012-special-release',
 '/spirits/thomas-h-handy-sazerac-rye-fall-2015',
 '/spirits/william-larue-weller-bourbon-fall-2015',
 '/spirits/the-balvenie-tun-1509-batch-1',
 '/spirits/booker-s-bourbon-batch-2015-01',
 '/spirits/four-roses-single-barrel-2013',
 '/spirits/macallan-rare-cask',
 '/spirits/four-roses-bourbon-single-barrel-2014',
 '/spirits/the-macallan-fine-oak-21',
 '/spirits/thomas-h-handy-sazerac-rye-fall-2014',
 '/spirits/parker-s-heritage-promise-of-hope',
 '/spirits/bunnahabhain-2008-moine-bordeaux-red-wine-cask-matured',
 '/spirits/bowmore-26-year-wine-matured',
 '/spirits/sazerac-rye-18-year-fall-2017',
 '/spirits/ardbeg-17-year',
 '/spirits/teeling-platinum-reserve-30-year-single-malt',
 '/spirits/hibiki-30-year',
 '/spirits/laphroaig-32-year',
 '/spirits/hibiki-21

In [85]:
df = pd.DataFrame(columns=['name', 'type', 'price', 'expert_score', 'user_rating', 'distillery_loc', 'description', 'age', 'abv', 'style', 'cask', 'review', 'smoky', 'peaty', 'spicy', 'herbal', 'oily', 'full-bodied', 'rich', 'sweet', 'briny', 'salty', 'vanilla', 'tart', 'fruity', 'floral'])
df

,name,type,price,expert_score,user_rating,distillery_loc,description,age,abv,style,...,oily,full-bodied,rich,sweet,briny,salty,vanilla,tart,fruity,floral


In [86]:
for link in tqdm(links):
    url = 'https://distiller.com' + link
    html_page = requests.get(url) 
    soup = BeautifulSoup(html_page.content, 'html.parser')
    
    try:
        name = soup.find('h1', {'class': 'name'})
        name = name.text    
    except AttributeError:
        continue
        
    try:
        wtype = soup.find('h2', {'class': 'type'})
        wtype = wtype.text
    except AttributeError:
        continue
        
    try:
        price = soup.find('li', {'class': 'cost'})
        price = price.find('div', {'data-behavior': 'cost-value'})
        price = price.get_attribute_list('class')
    except AttributeError:
        continue
        
    try:
        escore = soup.find('span', {'class': 'expert-rating'})
        escore = escore.text
    except AttributeError:
        continue
    try:
        rating = soup.find('span', {'itemprop': 'ratingValue'})
        rating = rating.text   
    except AttributeError:
        continue
        
    try:
        distillery_loc = soup.find('h2', {'class': 'location'})
        distillery_loc = distillery_loc.text
    except AttributeError:
        continue
        
    try:
        description =    soup.find('p', {'class': 'description'})
        description = description.text
    except AttributeError:
        continue
        
    try:
        age = soup.find('li', {'class': 'age'})
        age = age.find('div', {'class': 'value'})
        age = age.text
    except AttributeError:
        continue
    
    try:
        abv = soup.find('li', {'class': 'abv'})
        abv = abv.find('div', {'class': 'value'})
        abv = abv.text
    except AttributeError:
        continue
        
    try:
        style = soup.find('li', {'class': 'whiskey-style'})
        style = style.find('div', {'class': 'value'})
        style = style.text
    except AttributeError:
        continue
        
    try:
        cask = soup.find('li', {'class': 'cask-type'})
        cask = cask.find('div', {'class': 'value'})
        cask = cask.text
    except AttributeError:
        continue
        
    try:
        review = soup.find('p', {'itemprop': 'reviewBody'})
        review = review.text
    except AttributeError:
        continue
        
    try:
        flavor = soup.find('canvas', {'class': 'js-flavor-profile-chart'})
        flavor = flavor.get_attribute_list('data-flavors')
        flavor = ''.join(flavor)
        flavor = json.loads(flavor)
        smoky = flavor["smoky"]
        peaty = flavor["peaty"]
        spicy = flavor["spicy"]
        herbal = flavor["herbal"]
        oily = flavor["oily"]
        full_bodied = flavor["full_bodied"]
        rich = flavor["rich"]
        sweet = flavor["sweet"]
        briny = flavor["briny"]
        salty = flavor["salty"]
        vanilla = flavor["vanilla"]
        tart = flavor["tart"]
        fruity = flavor["fruity"]
        floral = flavor["floral"]
    except AttributeError:
        continue
        
    try:
        df = df.append({'name': name, 
                   'type': wtype, 
                   'price': price, 
                   'expert_score': escore, 
                   'user_rating': rating, 
                   'distillery_loc': distillery_loc, 
                   'description': description, 
                   'age': age, 
                   'abv': abv, 
                   'style': style, 
                   'cask': cask, 
                   'review': review, 
                   'smoky': smoky, 
                   'peaty': smoky, 
                   'spicy': spicy, 
                   'herbal': herbal, 
                   'oily': oily, 
                   'full-bodied': full_bodied, 
                   'rich': rich, 
                   'sweet': sweet, 
                   'briny': briny, 
                   'salty': salty, 
                   'vanilla': vanilla, 
                   'tart': tart, 
                   'fruity': fruity, 
                   'floral': floral
                  }, ignore_index=True)
    except AttributeError:
        continue

In [87]:
df

,name,type,price,expert_score,user_rating,distillery_loc,description,age,abv,style,...,oily,full-bodied,rich,sweet,briny,salty,vanilla,tart,fruity,floral
0,\nHibiki 21 Year\n,Blended,"[cost-5, spirit-cost]",\n99\n\n,4.52,\nSuntory // Japan\n,Hibiki is the blended whisky brand produced by...,21 Year,43.0,Comprised of both single malt(s) and single gr...,...,20,80,80,85,0,15,20,25,85,50
1,\nHighland Park 18 Year\n,Peated Single Malt,"[cost-4, spirit-cost]",\n99\n\n,4.49,"\nHighland Park // Islands, Scotland\n",Shudder to think that Highland Park 18 has onl...,18 Year,43.0,100% malted barley from a single distillery. D...,...,40,70,80,70,20,40,50,50,70,20
2,\nGeorge T. Stagg Bourbon (Fall 2019)\n,Bourbon,"[cost-4, spirit-cost]",\n98\n\n,4.59,"\nGeorge T. Stagg // Kentucky, USA\n",George T. Stagg Bourbon (Fall 2019) is an uncu...,NAS,58.45,Produced anywhere in USA; Mash bill of at leas...,...,20,60,60,45,0,0,60,60,45,0
3,\nBowmore Mizunara Cask Finish\n,Peated Single Malt,"[cost-5, spirit-cost]",\n98\n\n,4.19,"\nBowmore // Islay, Scotland\n",This is the first-ever official Islay single m...,NAS,53.9,100% malted barley from a single distillery. D...,...,10,75,75,60,15,20,30,20,50,5
4,\nThe Macallan M\n,Single Malt,"[cost-5, spirit-cost]",\n98\n\n,4.19,"\nThe Macallan // Highlands, Scotland\n",A 6-litre Lalique decanter of this single malt...,NAS,47.7,Distilled in pot stills from 100% malted barle...,...,15,90,100,50,0,0,20,20,80,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2823,\nWild Turkey Spiced\n,Flavored Whiskey,"[cost-1, spirit-cost]",\n59\n\n,2.93,"\nWild Turkey // Kentucky, USA\n",Wild Turkey Spiced is an 86-proof bourbon flav...,NAS,43.0,Whiskey flavored with natural or artificial fl...,...,0,40,55,85,0,0,25,10,10,0
2824,\nSeagram's 7 Crown American Blended Whiskey\n,Blended American Whiskey,"[cost-1, spirit-cost]",\n59\n\n,2.08,\nSeagram's // USA\n,Originally a Canadian whisky brand until it wa...,NAS,40.0,Straight whiskey(s) must comprise at least 20%...,...,0,30,0,100,0,0,100,0,60,10
2825,\n11 Wells Single Malt Whiskey\n,American Single Malt,"[cost-3, spirit-cost]",\n58\n\n,3.0,"\n11 Wells // Minnesota, USA\n",This Single Malt release from 11 Wells is part...,5 Months,42.0,Made in the USA from 100% malted barley. Disti...,...,40,10,10,90,0,0,50,20,30,10
2826,\nImmortal Spirits Early Whiskey\n,Other Whiskey,"[cost-2, spirit-cost]",\n50\n\n,1.0,"\nImmortal Spirits // Oregon, USA\n",Distilled from barley grown in the Rogue Valle...,NAS,44.5,A spirit distilled from a mash of grain(s) at ...,...,80,0,0,40,0,0,20,0,0,0


In [88]:
df.describe()

,name,type,price,expert_score,user_rating,distillery_loc,description,age,abv,style,...,oily,full-bodied,rich,sweet,briny,salty,vanilla,tart,fruity,floral
count,2828,2828,2828,2828,2828,2828,2828,2828,2828,2828,...,2828,2828,2828,2828,2828,2828,2828,2828,2828,2828
unique,2805,21,5,41,269,1027,2803,175,304,21,...,45,45,47,48,45,46,49,50,46,54
top,\nLaphroaig 32 Year \n,Single Malt,"[cost-2, spirit-cost]",\n87\n\n,,\nNikka // Japan\n,This is the high proof rye of the 5 whiskeys k...,NAS,40.0,Distilled in pot stills from 100% malted barle...,...,20,60,60,60,0,0,40,0,40,0
freq,2,715,858,239,82,32,2,1581,518,715,...,428,469,442,451,1631,1381,401,466,304,564


In [89]:
df.to_csv("whisky_df.csv")